# Python Skript für Dynamische Diagramme zur Anzeige von SAR-Parametern für die Überwachung von landwirtschaftlichen Feldern.

### Der Zweck dieses Tools ist die kohärente Darstellung mehrerer SAR-Parameter im Zeitverlauf in Verbindung mit Hilfsdaten wie zum Beispiel Wetterdaten. Diese Darstellung soll als Web-App interaktiv gestaltet werden, wofür Streamlit verwendet wurde. Insgesamt werden drei Untersuchungsgebiete in Deutschland abgedeckt. 

### Ersteller: Markus Adam & Laura Walder


Zunächst galt es die entsprechend benötigten Pakete in Python zu installieren, wozu streamlit, pandas, sqlite3 und altair gehören. Anschließend wurde eine Verbindung zu der bereits existierenden Datenbank hergestellt.

In [ ]:
import streamlit as st
import pandas as pd
import sqlite3
import altair as alt

# connection to db
try:
    db = sqlite3.connect(
        'C:/Users/Laura/Desktop/UNI/2.Semester/Python_Teil_II/Abschlussprojekt/students_db_sql_queries_3/RCM_work.db')
    cursor = db.cursor()
    print("Successfully Connected to SQLite Database")
except sqlite3.Error as error:
    print("Error while connecting to Database", error)

Anschließend kann die gewüschte resultierende Web Ansicht konfiguriert werden.
Hier wurde sich für ein weites Layout entschieden, sodass die resutierenden Graphen möglichst groß dargestellt werden.
Die ensprechenden Python Befehle sind mit streamlit durchgeführt worden und sind durch das vorgestellte "st." zu erkennen.
Bei "st.titel" und "st.markdown" handelt es sich um Print-Befehle mit unterschiedlichen Schriftgrößen:
## st.titel
### st.markdown
Mithilfe von st.markdown('#') wird eine leere Textzeile eingefügt, wodurch ein Absatz erzeugt wird.

In [ ]:
# set page layout
st.set_page_config(layout="wide")

# create title and description
st.title('Radar Crop Monitor App')
st.markdown('This app can be used to display SAR parameters and NDVI values for crop monitoring.')
st.markdown("Please select the main and dependent filter first and note that displaying the data may take some time.")
st.markdown('#')

### Hauptfilter
Die Area of Interest (AOI), das Jahr, der Anpflanzungstyp und die gewählte Statistik gilt es als Hauptfilter zu integrieren. 
Hierfür werden die benötigten Informationen zunächst aus den jeweiligen Spalten aus den verschiedenen Tabellen entnommen und mit entsprechenden Namen definiert. 
Dies geschieht mit einer "Datenbankabfrage" und dem Befehl "pd.read_sql_query".
Die abhängigen Filter (siehe unten) werden hier ebenfalls schon abgefragt.
Durch die SQL Abfrage "select distinct" wird sichergestellt, dass Werte (auch wenn sie sich häufig wiederholen) nur einmal eingeladen werden. 

In [ ]:
# create titles for data filters
st.sidebar.title("Filters")
st.sidebar.markdown("#")
st.sidebar.header('Main Filters')

# load values from specific table columns for value selection (filters) by user
aoi_names = pd.read_sql_query('select distinct aoi from areaofinterest;', db)
years = pd.read_sql_query("select distinct year from areaofinterest;", db)
crop_types = pd.read_sql_query("select distinct crop_type from croplegend;", db)
products = pd.read_sql_query("select distinct product from s1fieldstatistic;", db)
acq_types = pd.read_sql_query("select distinct acquisition from s1fieldstatistic;", db)
parameter = pd.read_sql_query("select distinct polarization from s1fieldstatistic;", db)
stats = pd.read_sql_query("select distinct statistic from s1fieldstatistic;", db)
fid = pd.read_sql_query("select distinct fid from areaofinterest;", db)

Anschließend können die vier Hauptfilter erstellt werden.
Hier wurde sich für ein Drop-Down-Menü entschieden, bei dem jeweils nur ein Wert gewählt werden kann.
Dies ist in Streanlit als eine selectbox definiert, in welcher zugleich die Überschrift festgelegt wird.

In [ ]:
# get single value selections from user
aoi_selection = st.sidebar.selectbox("AOI", aoi_names)
year_selection = st.sidebar.selectbox("Year", years)
crop_selection = st.sidebar.selectbox("Crop Type", crop_types)
stat_selection = st.sidebar.selectbox("Statistic", stats)

### Abhängige Filter
Ähnlich wie bei den Haupfiltern werden nun die abhängigen Filter erstellt.
Da hier jedoch eine Mehrfachauswahl möglich sein soll wird eine "mulitselection" gewählt. 

In [ ]:
st.sidebar.markdown('#')
st.sidebar.header('Dependent Filters')

# get list of multiselections from user
acq_selection = tuple(st.sidebar.multiselect("Acquisition Mode", acq_types))
product_selection = tuple(st.sidebar.multiselect("Product", products))
param_selection = tuple(st.sidebar.multiselect("Parameter", parameter))
fid_selection = tuple(st.sidebar.multiselect("FID", fid))


Letztlich werden in den Seitenteil der Web App unter die Filter noch die Datenquellen und die Erschaffer erwähnt:

In [ ]:
# print data source and contributors
st.sidebar.markdown("#")
st.sidebar.markdown("Data Source: ESA Copernicus-Data")
st.sidebar.markdown("Contributors: Markus Adam, Laura Walder")

Von den Multiselektionen (das heißt von den abhängigen Filtern) wird anschließend eine Liste erstellt um sie zusammenzufassen.
# ????????Grund für Placeholder!!!!!!!!!!!!

In [ ]:
# list of multiselections
dependent_selections = [acq_selection, product_selection, param_selection, fid_selection]

# function to add placeholder to multiselection tuple if len == 1 (prevents syntax error)

def placeholders(multiselections):
    if len(multiselections) == 1:
        multiselections = multiselections + ("placeholder",)
        return multiselections
    else:
        return multiselections

# apply placeholder function
acq_selection = placeholders(acq_selection)
param_selection = placeholders(param_selection)
product_selection = placeholders(product_selection)
fid_selection = placeholders(fid_selection)


### SQL Körper
Um dem User nun die Informationen aus der Datenbank zur Auswahl anzubieten, müssen diese zunächst als SQL Abfrage eingeladen werden. Im unteren Teil werden zudem, die benötigten Informationen für die gewählten Filter eingeladen und umbenannt.

In [ ]:
# define sql body
sql = f"""SELECT 
    round(s1.value, 2) as value, 
    s1.mask_label, 
    s1.unit, 
    s1.aoi, 
    s1.datetime, 
    strftime('%Y-%m-%d', s1.datetime) as date,
    strftime('%H:%M:%S', s1.datetime) as time,
    s1.polarization as parameter, 
    s1.acquisition, 
    s1.product,
    area.fid, 
    area.year, 
    area.sl_nr, 
    area.crop_type_code, 
    area.crop_type,
    area.field_geom
    FROM s1fieldstatistic as s1
    INNER JOIN (SELECT 
    areaofinterest.fid, 
    areaofinterest.year, 
    areaofinterest.aoi,
    areaofinterest.sl_nr, 
    areaofinterest.crop_type_code, 
    crop.crop_type,
    areaofinterest.field_geom
    FROM areaofinterest
    INNER JOIN croplegend as crop 
    ON (crop.crop_type_code = areaofinterest.crop_type_code)) area
    ON (s1.mask_label = area.fid AND strftime('%Y', s1.datetime)=area.year AND s1.aoi = area.aoi)
    WHERE 
    s1.aoi="{aoi_selection}"
    AND area.crop_type="{crop_selection}"
    AND area.year="{year_selection}"
    AND s1.product IN {repr(product_selection)}
    AND s1.acquisition IN {repr(acq_selection)}
    AND s1.polarization IN {repr(param_selection)}
    AND area.fid IN {repr(fid_selection)}
    AND s1.statistic = "{stat_selection}"
    ORDER BY s1.mask_label, s1.datetime  ASC; """


Anschließend werden die Records aus einer Abfrage der Datenbank und dem zuvor definierten SQL Körper definiert

In [ ]:
# load table as df with sql query
records = pd.read_sql(sql, db)

Im folgenden wurden zudem Warnungen und Fehlermeldungen definiert.
Es soll zu Beginn direkt eine Warnung erscheinen, die den User auffordert Daten auszuwählen, sodass eine entsprechende Graphik dazu erstellt werden kann.
Da die Daten jedoch begrenzt sind, sind nicht alle Kombinationen aus den Filtern in den Daten vorhanden, weshalb dem User eine Fehlermeldung mitgeteilt wird, sollte eine Kombination der Filter gewählt worden sein, welche keine Informaionen beinhalten. 

In [ ]:
# print warning when no filter is selected and error when invalid filter combination (with no data) is selected
if records.empty:
    if all(len(x) == 0 for x in dependent_selections):
        st.warning("No selection has been made. Please select filter combinations")
    else:
        st.error("No data is available with this filter combination. Please select other filter combinations")

Um zudem die Web App optisch zu optimieren wurde ein Expander integriert, durch welchen der Zeitfilter, sowie die Auswahl der Trendlinien "eingeklappt" werden kann, wodurch die Graphen weiter nach oben rutschen und die Filter nicht weiter stören.
Das heißt alles Folgende, welches mit "expander." beginnt, wird in der Web App "einklappbar" sein.

In [ ]:
# define expander box for time slider and trendline selection
expander = st.expander("Time and Trendline Filter", expanded=True)

### Zeitfilter
Anschließend gilt es den Zeitfilter zu integrieren. 
Hierfür wurde die bereits existierende Spalte "datetime" aus der Datenbank abgerufen.
Mit den Befehlen "min" und "max" lassen sich nun das frühste und späteste Datum des Datensatzes finden, welche als "Startdatum" und "Enddatum" definert werden.

In [ ]:
# create sliders for time frame selection (start and end date of time series plots)
# convert datetime string column to datetime
records["datetime"] = pd.to_datetime(records["datetime"])
records["datetime"] = pd.to_datetime(records['datetime']).apply(lambda x: x.date())

# get earliest and latest date from df as boundaries for slider
start_date = records["datetime"].min()
end_date = records["datetime"].max()

Dem Zeitfilter werden nun die Eingaben des Users zugeordnen, ein entsprechendes Format (TT.MM.JJ) wird gewählt und die Label bestimmt. Auch hier wird eine Warnung integriert, welche in der Web App erscheint, sobald sie geöffnet wird, da zunächste eine Auswahl der Filter getroffen werden muss, woraufhin dann der bereits gefilterte Datensatz erneut zeitlich gefiltert werden kann.

In [ ]:
# define slider values from user selection and filter df based on these values
try:
    expander.subheader("Select date range")
    slider_1, slider_2 = expander.slider('', value=(start_date, end_date), format="DD.MM.YY")
    records = records[(records['datetime'] > slider_1) & (records['datetime'] < slider_2)]
except KeyError:
    expander.warning("Date range slider is only available after a valid filter combination has been selected")

Sobald die Filer von dem User gesetzt wurden, steht der Zeit-Slider zur Verfügung.
Start- und Enddatum werden daher nun, abhängig von der Auswahl des Users, neu definiert.

In [ ]:
# get earliest and latest date again from now date-filtered df
start_date = records["datetime"].min()
end_date = records["datetime"].max()

Anschließend werden dem User statistische Trendlinien zur Auswahl gegeben, wofür im Folgenden die "Auswahlknöpfe" erstellt werden. 

In [ ]:
# make button for selection of trend line type
expander.markdown("#")
expander.subheader("Select statistic trendline for the graphs")
stat_button = expander.radio("", ("None", "LOESS", "Rolling Mean"))
st.markdown("#")

Der gefilterte Datensatz soll nun in Graphen dargestellt werden.
Hierfür wurde sich für eine Differnzierung nach der Polarisation (VV/VH) bzw. den NDVI Werten entschieden.
Entsprechend wurden diese zunächst aus den Records definiert:

In [ ]:
# filter df by polarisation/value for different plots
vv_records = records[records["parameter"] == "VV"]
vh_records = records[records["parameter"] == "VH"]
ndvi_records = records[records["parameter"] == "NDVI"]


# ?????

In [ ]:
# set df column by which points are colored
selection = alt.selection_multi(fields=['acquisition'], bind='legend')

# set domain containing earliest and latest date of dataset, used as boundaries for x-axis of charts
domain_pd = pd.to_datetime([start_date, end_date]).astype(int) / 10 ** 6

### Erstellen der Graphen
Schlussendlich können nun die Graphen erstellt werden.
Hier werden die Informationen den Achsen zugeordnet, wobei hier die Zeit auf der X-Achse und die Rückstreuuwerte auf der Y-Achse vertreten sind. Zudem werden die Label, wie die Überschrift und die Achsenbeschriftungen, sowie die Höhe und Breite der Graphik bestimmt.

In [ ]:
# VV polarization charts
vv_chart = alt.Chart(vv_records).mark_circle().encode(
    x=alt.X("datetime:T", axis=alt.Axis(title='Date', titleFontSize=22), scale=alt.Scale(domain=list(domain_pd))),
    y=alt.Y("value", axis=alt.Axis(title='Backscatter', titleFontSize=22)),
    color=alt.condition(selection, "acquisition", alt.value("lightgray"), sort=["D"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))).add_selection(selection).\
    properties(title="VV Polarization", width=1000, height=500)

Anschließend wird die Loess Trendlinie in die Graphik integriert, sollte der entsprechende Button vom User gewählt werden. Der entsprechende Befehl "transform_loess" ist von Streamlit bekannt. 

In [ ]:
vv_loess = alt.Chart(vv_records).encode(
    x=alt.X("datetime:T", axis=alt.Axis(title='Date', titleFontSize=22), scale=alt.Scale(domain=list(domain_pd))),
    y=alt.Y("value", axis=alt.Axis(title='Backscatter', titleFontSize=22))).transform_filter(selection).\
    transform_loess("datetime", "value").mark_line(color="black")

Ähnliches wird für die Mittelwert Trendlinie durchgeführt:

In [ ]:
vv_mean = alt.Chart(vv_records).mark_line(color="black").transform_filter(selection).\
    transform_window(rolling_mean="mean(value)", frame=[-5, 5]).encode(x='datetime:T', y='rolling_mean:Q')

Das gleiche (Erstellung der Graphen, Zufügen der Trendlinien) wird nun für die VH Polarisation erstellt:

In [ ]:
# VH polarization charts
vh_chart = alt.Chart(vh_records).mark_circle().encode(
    x=alt.X("datetime:T", axis=alt.Axis(title='Date', titleFontSize=22), scale=alt.Scale(domain=list(domain_pd))),
    y=alt.Y("value", axis=alt.Axis(title='Backscatter', titleFontSize=22)),
    color=alt.condition(selection, "acquisition", alt.value("lightgray"), sort=["D"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))).add_selection(selection).\
    properties(title="VH Polarization", width=1000, height=500)
vh_loess = alt.Chart(vh_records).encode(
    x=alt.X("datetime:T", axis=alt.Axis(title='Date', titleFontSize=22), scale=alt.Scale(domain=list(domain_pd))),
    y=alt.Y("value", axis=alt.Axis(title='Backscatter', titleFontSize=22))).transform_filter(selection).\
    transform_loess("datetime", "value").mark_line(color="black")
vh_mean = alt.Chart(vh_records).mark_line(color="black").transform_filter(selection).\
    transform_window(rolling_mean="mean(value)", frame=[-5, 5]).encode(x='datetime:T', y='rolling_mean:Q')

Und erneut für die NDVI Werte:

In [ ]:
# NDVI charts
ndvi_chart = alt.Chart(ndvi_records).mark_circle().encode(
    x=alt.X("datetime:T", axis=alt.Axis(title='Date', titleFontSize=22), scale=alt.Scale(domain=list(domain_pd))),
    y=alt.Y("value", axis=alt.Axis(title='NDVI Value', titleFontSize=22)),
    color=alt.condition(selection, "acquisition", alt.value("lightgray"), sort=["D"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))).add_selection(selection).\
    properties(title="NDVI", width=1000, height=500)
ndvi_loess = alt.Chart(ndvi_records).encode(
    x=alt.X("datetime:T", axis=alt.Axis(title='Date', titleFontSize=22), scale=alt.Scale(domain=list(domain_pd))),
    y=alt.Y("value", axis=alt.Axis(title='NDVI Value', titleFontSize=22))).transform_filter(selection).\
    transform_loess("datetime", "value").mark_line(color="black")
ndvi_mean = alt.Chart(ndvi_records).mark_line(color="black").transform_filter(selection).\
    transform_window(rolling_mean="mean(value)", frame=[-5, 5]).encode(x='datetime:T', y='rolling_mean:Q')

### Statistische Trendlinien einfügen
Die oben erstellten Trendlinien müssen nun noch mit den Auswahl-Knöpfen verknüpft werden:

In [ ]:
# set trend line type in charts based on user selection
if stat_button == "LOESS":
    vv_chart = vv_chart + vv_loess
    vh_chart = vh_chart + vh_loess
    ndvi_chart = ndvi_chart + ndvi_loess

elif stat_button == "Rolling Mean":
    vv_chart = vv_chart + vv_mean
    vh_chart = vh_chart + vh_mean
    ndvi_chart = ndvi_chart + ndvi_mean

Letztlich sollen natürlich nur die Graphen visulaiert werden, welche vom User ausgwählt worden sind, das heißt die Graphen sollen nur erzeugt werden, sobald die entsprechenden Parameter (VV, VH oder NDVI) in der Auwahl enthalten sind:

In [ ]:
# define list of charts to be displayed, based on user selection and data availability
chart_list = []
if records["parameter"].str.contains("VV").any() and "VV" in param_selection:
    chart_list.append(vv_chart)
if records["parameter"].str.contains("VH").any() and "VH" in param_selection:
    chart_list.append(vh_chart)
if records["parameter"].str.contains("NDVI").any() and "NDVI" in param_selection:
    chart_list.append(ndvi_chart)

Die resultierende Liste an Graphen werden im Folgenden nun final mit Hife von Altair visualisiert, wobei erneut Titel und Schriftgröße bestimmt werden.

In [ ]:
# display charts from list
for chart in chart_list:
    st.altair_chart(chart.configure_title(fontSize=28).configure_legend(titleFontSize=20, labelFontSize=18))

Nach der Erzeugung der Graphen kann die Verbindung zu der Datenbank wieder getrennt werden:

In [ ]:
# close connection to db
cursor.close()
db.close()

Nachdem das Skript vollständig erfolgreich durchgelaufen hat, bekommt man bei Python folgende Meldung:

"Warning: to view this Streamlit app on a browser, run it with the following command: 
streamlit run C:/Users/Laura/Desktop/UNI/2.Semester/Python_Teil_II/Abschlussprojekt/main2.py"

Gibt man nun, wie aufgefordert, diesen Befehl in das Terminal ein, öffnet sich erfolgreich die Web App im Browser.